# Imports

In [1]:
import numpy as np
import pandas as pd
import progressbar as pb
from custom_modules.helper_functions import *

# Define Read Function

In [2]:
def filepart_str(filename_base, part_num, curr=None):
    """
    Return a formatted filename string from a base string and a part
    number.
    
    filename_base : str
        The base string of the name of the data to be loaded.
    part_num : int
        The part number included in the name of the data to be loaded.
    curr : "local" or "usd", default None
        If provided, indicates that a choice of currency folder should
        be included in the file directory string.
    """
    
    if curr is None:
        full_base = filename_base
    elif curr == "usd":
        full_base = "usd-returns\\"+filename_base
    elif curr == "local":
        full_base = "local-returns\\"+filename_base
    else:
        raise ValueError("curr must be 'usd' or 'local'.")
    
    output = (
        "..\\Data\\Raw Data\\Mutual Funds\\{base}\\"
         "mf_{base}_part-{part}.csv".format(base=full_base, part=part_num)
    )
    
    return output

In [3]:
def dfread(filename_base, exp_dtype=None, thousands=None, curr=None):
    """
    Read all parts of a multi-part dataset by looping through part
    numbers within a given directory.

    Parameters
    ----------
    filename_base : str
        The base string of the name of the data to be loaded.
    exp_dtype : type, default None
        If provided, sets the datatype of all columns to be loaded
        except for the three left-most columns.
    thousands : str, default None
        If provided, indicates a thousands separator in the data to be
        read.
    curr : "local" or "usd", default None
        If provided, indicates that a choice of currency folder should
        be included in the file directory string.

    Returns
    -------
    output : pd.DataFrame
        A loaded dataframe containing all parts of the data present in
        the given directory
    """
        
    # --- SET UP ---
    # If any explicit datatypes are given, pull out the column names
    # to declare which columns should be typed in the next read
    if exp_dtype is not None:
        col_names = pd.read_csv(filepart_str(filename_base, 1),
                                nrows=0).columns
        dict_dtypes = dict(zip(col_names[3:],
                               [exp_dtype]*(col_names.size-3)))
    else:
        dict_dtypes = None
        
    # Add a folder for the choice of currency if necessary
    if curr is None:
        full_base = filename_base
    elif curr == "usd":
        full_base = "usd-returns\\"+filename_base
    elif curr == "local":
        full_base = "local-returns\\"+filename_base
    else:
        raise ValueError("curr must be 'usd' or 'local'.")
        
    # --- READ FILES ---
    # Read part one of the base data
    try:
        df_return = pd.read_csv(filepart_str(filename_base, 1, curr=curr),
                                dtype=dict_dtypes, thousands=thousands)
    except FileNotFoundError:
        raise ValueError("{} is an invalid filename_base (no part "
                         "one file in directory)".format(filename_base))

    # Initialise loop variables
    i = 2  # Start on part 2
    loading = True

    while(loading):
        try:
            df_concat = pd.read_csv(filepart_str(filename_base, i, curr=curr),
                                    dtype=dict_dtypes, thousands=thousands)
        except FileNotFoundError:
            # If the next part doesn't exist, turn off the loop
            loading = False
        else:
            # Only concatenate again if new data was found
            df_return = pd.concat([df_return, df_concat])
            i += 1

    return df_return.reset_index(drop=True)

# Read Country Information

Read the info dataframe from its parts, select only the columns for secid and country, and count the number of funds domiciled in each country

In [5]:
df_countries = dfread("info").loc[:,"SecId":"Domicile"]

C:\Users\ozboy\AppData\Local\Temp\ipykernel_13116\1140695005.py:63: DtypeWarning: Columns (313,344,357,384) have mixed types. Specify dtype option on import or set low_memory=False.
  df_concat = pd.read_csv(filepart_str(filename_base, i, curr=curr),


In [42]:
fundcount = (
    df_countries.groupby("Domicile").SecId.count().sort_values(ascending=False)
)

print("The total number of funds is: {}".format(fundcount.sum()))
print("Display fundcount for a breakdown of funds domiciled in each country.")

The total number of funds is: 569375
Display fundcount for a breakdown of funds domiciled in each country.


# Group countries

Form groups of countries in roughly descending order of fund count such that every country is included in at most one group and every group has at most 65,000 funds in it unless it contains only one country.

In [6]:
country_groups = {
    "lux": ["Luxembourg"],
    "kor": ["South Korea"],
    "usa": ["United States"],
    "can-chn-jpn": ["Canada", "China", "Japan"],
    "irl-bra": ["Ireland", "Brazil"],
    "gbr-fra-ind": ["United Kingdom", "France", "India"],
    "esp-tha-aus-zaf-mex-aut-che": [
        "Spain", "Thailand", "Australia", "South Africa", "Mexico", "Austria",
        "Switzerland"
    ],
    "other": [
        "Belgium", "Germany", "Chile", "Italy", "Taiwan", "Israel",
        "Cayman Islands", "Liechtenstein", "Sweden", "Finland", "Hong Kong",
        "Malaysia", "Denmark", "Guernsey", "Netherlands", "Norway", "Singapore",
        "Indonesia", "New Zealand", "Jersey", "Malta", "Portugal", "Poland",
        "Bermuda", "British Virgin Islands", "Hungary", "Mauritius", "Turkey",
        "Saudi Arabia", "Philippines", "Namibia", "Greece", "Isle of Man",
        "Andorra", "Slovenia", "Russian Federation", "Bahamas", "Iceland",
        "Estonia", "Kuwait", "United Arab Emirates", "Curaçao", "Bahrain",
        "Botswana", "Lithuania", "Latvia", "Czech Republic", "Monaco",
        "Puerto Rico", "Gibraltar", "Colombia", "Argentina", "San Marino",
        "St Vincent-Grenadines", "Oman", "Swaziland", "Lesotho", "Viet Nam",
        "Barbados", "Kenya", "Lebanon", "Jordan", "Qatar", "Tunisia", "Samoa",
        "US Virgin Islands", "Bulgaria", "Uganda", "Cyprus",
        "Marshall Islands", "Netherlands Antilles", "Panama", "Ukraine",
        "Pakistan", "Uruguay", "Vanuatu", "Peru"
    ]
}

In [7]:
group_secids = dict()
for i in country_groups:
    group_mask = df_countries.Domicile.isin(country_groups[i])
    group_secids[i] = df_countries.loc[group_mask, "SecId"].values

# Read and Split Data

Define a function to read each dataframe from its parts, partition that frame across countries, and then resave using the new partition

In [8]:
def countrysplit(filename_base, exp_dtype=None, thousands=None, curr=None):
    """
    Read each combined DataFrame from its consituent parts, partition
    that DataFrame across countries and resave using the new partition.

    Parameters
    ----------
    filename_base : str
        The base string of the name of the data to be loaded.
    exp_dtype : type, default None
        If provided, sets the datatype of all columns to be loaded
        except for the three left-most columns.
    thousands : str, default None
        If provided, indicates a thousands separator in the data to be
        read.
    curr : "local" or "usd", default None
        If provided, indicates that a choice of currency folder should
        be included in the file directory string.
    """
    
    # Start a progress bar.
    bar = pb.ProgressBar(max_value = len(group_secids)+1)
    bar.update(0)
        
    # Load the full set of data and update the progress bar once.
    df_active = dfread(filename_base, exp_dtype, thousands, curr)
    bar.update(1)

    
    # Add a folder for the choice of currency if necessary
    if curr is None:
        full_base = filename_base
    elif curr == "usd":
        full_base = "usd-returns\\"+filename_base
    elif curr == "local":
        full_base = "local-returns\\"+filename_base
    else:
        raise ValueError("curr must be 'usd' or 'local'.")
        
    # For all groups except "other", split off only those funds that
    # come from countries belonging to that group, but for the "other"
    # group also include any funds that have no domicile.
    for i in group_secids:
        dom_mask = df_active.SecId.isin(group_secids[i])
        nondom_mask = ((i == "other")
                       & ~(df_active.SecId.isin(df_countries.SecId)))
        
        df_split = (df_active[dom_mask | nondom_mask].copy())
        
        df_split.to_csv("..\\Data\\Raw Data\\Mutual Funds - Country Grouped\\"
                         "{fbase}\\mf_{base}_{i}.csv"
                        .format(fbase=full_base, base=filename_base, i=i),
                        index=False)
        bar.update(bar.value+1)
        
    return

Function calls

In [31]:
countrysplit("info")

100% (9 of 9) |##########################| Elapsed Time: 0:00:07 ETA:  00:00:00

In [32]:
countrysplit("monthly-costs", np.float64, ",")

100% (9 of 9) |##########################| Elapsed Time: 0:02:45 ETA:  00:00:00

In [46]:
countrysplit("monthly-gross-returns", np.float64, ",", "usd")

N/A% (0 of 9) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

FileNotFoundError: [Errno 2] No such file or directory: '..\\Data\\Raw Data\\Mutual Funds\\monthly-gross-returns\\mf_monthly-gross-returns_part-1.csv'

In [ ]:
countrysplit("monthly-gross-returns", np.float64, ",", "local")

In [ ]:
countrysplit("monthly-net-returns", np.float64, ",", "usd")

In [ ]:
countrysplit("monthly-net-returns", np.float64, ",", "local")

In [9]:
countrysplit("monthly-morningstar-category", object)

100% (9 of 9) |##########################| Elapsed Time: 0:01:25 ETA:  00:00:00

In [ ]:
countrysplit("monthly-net-assets", np.float64, ",")